<a href="https://colab.research.google.com/github/jchen6727/pubtk/blob/development/examples/colab_driveless/pubtk2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tutorial 2**

**Note 0** This tutorial will demonstrate using the `dispatcher`<->`runner` pair from tutorial 2 with socket communication features to run a group of simulations


In [ ]:
#jupyter 0
!git clone --depth 1 --single-branch --branch development https://github.com/jchen6727/pubtk.git /content/pubtk # clone from the development branch
!pip install -e /content/pubtk
import site
site.addsitedir('/usr/local/lib/python3.10/dist-packages')
!curl https://raw.githubusercontent.com/jchen6727/pubtk/development/examples/colab/socket_runner.py > /content/runner.py

**Note 1** This time we'll use the **INETDispatcher** which uses `INET sockets` (TCP) to communicate with the host. Let's inherit the `INETDispatcher` as well as recreate our Submit from pubtk2

In [ ]:
from pubtk.runtk import INETDispatcher, Submit, Template
#jupyter 1
class GCSubmitSOCK(Submit):
  def __init__(self):
    # creates a Submit with the templates we define
    super().__init__(
        submit_template = Template("sh {output_path}/{label}.sh"),
        script_template = Template("""\
#!/bin/bash
cd {project_path}
export SOCNAME="{sockname}"
{env}
nohup python /content/runner.py > {output_path}/{label}.run 2>&1 &
pid=$!
echo $pid >&1
"""
        )
    )
  def submit_job(self):
    # using this submit_job, we can add some handling of stdout, job failure (i.e. if stdout does not return an integer value as expected),
    # extending the functionality of Submit with this exception handling.
    proc = super().submit_job()
    try:
      self.job_id = int(proc.stdout)
    except Exception as e:
      raise(Exception("{}\nJob submission failed:\n{}\n{}\n{}\n{}".format(e, self.submit, self.script, proc.stdout, proc.stderr)))
    return self.job_id

gcs = GCSubmitSOCK()

**Note 2**
Now we'll create functions wrapping the `dispatcher->submit->receive` calls.
Let's also create a series of job configurations to run

In [ ]:
#jupyter 2
def start_job(config, index):
  dispatcher = INETDispatcher(project_path='/content', output_path='./batch', submit=gcs, gid='batch_example{}'.format(index))
  dispatcher.update_env(config)
  dispatcher.create_job()
  dispatcher.submit_job()
  return dispatcher

def recv_job(dispatcher):
  connection, runner_address = dispatcher.accept()
  print( (connection, runner_address) )
  message = dispatcher.recv()
  dispatcher.clean()
  return message


configurations = [{'x': i, 'y': j, 'z': k} for i, j, k in zip(range(1,4), range(4,7), range(7,10))]
print(configurations)

**Note 3** Now we can use `for` loops to create a series of jobs and aggregate the outputs.

In [ ]:
#jupyter 3
dispatchers = []
for index, config in enumerate(configurations):
  dispatchers.append(start_job(config, index))

messages = []
for dispatcher in dispatchers:
  messages.append(recv_job(dispatcher))

print(messages)